In [1]:
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.display import display
from IPython.utils.capture import capture_output
from pprint import pprint
from pathlib import Path
import re
from base64 import b64decode

In [2]:
@magics_class
class CaptureMagic(Magics):
    @magic_arguments.magic_arguments()
    @magic_arguments.argument(
        "--path",
        "-p",
        default=None,
        help=(
            "The path where the video will be saved to. When there is more then one video, multiple paths have to be defined"
        ),
    )
    @cell_magic
    def capture_video(self, line, cell):
        args = magic_arguments.parse_argstring(CaptureMagic.capture_video, line)
        paths_pathlib = args.path.strip('"').split(" ")
        with capture_output(stdout=False, stderr=False, display=True) as result:
            self.shell.run_cell(cell)
        for output in result.outputs:
            display(output)  # only disabled for debugging
            global data  # for debugging
            data = output.data

            pprint(data)  # for debugging

            print("#####")

            if "text/html" in data:
                path = paths_pathlib.pop(0)
                if not path:
                    raise ValueError("Too few paths given!")
                video_object_html_string = data["text/html"]

                # Extract video source filename
                pattern = re.compile(r'video src="(.+?)"')
                match = pattern.search(video_object_html_string)

                if match:
                    video_dir = match.group(1)
                    print(f"Found video at {video_dir}")

                    dest = Path(path)
                    src = Path(video_dir)
                    dest.write_bytes(src.read_bytes())

                else: # debugging only
                    print("No video source found in HTML string.")

                # Extract base64-encoded embedded video data 
                pattern = re.compile(r'data:video/mp4;base64,(.*)">')
                match = pattern.search(video_object_html_string)

                if match:
                    base64_string = match.group(1)
                    print("Found base64-encoded video")

                    # Decode base64 string and save video file using pathlib
                    video_bytes = b64decode(base64_string)
                    assert isinstance(video_bytes, bytes)
                    dest = Path(path)
                    dest.write_bytes(video_bytes)

                else:
                    print("No base64-encoded video data found in HTML string.")



ipy = get_ipython()
ipy.register_magics(CaptureMagic)


In [3]:
from IPython.display import Video

In [4]:
%%capture_video -p "not_embedded.mp4"
# ✅✅✅
Video("../assets/dog_with_water.mp4", embed=False, width=300)

{'text/html': '<video src="../assets/dog_with_water.mp4" controls  width="300" '
              '>\n'
              '      Your browser does not support the <code>video</code> '
              'element.\n'
              '    </video>',
 'text/plain': '<IPython.core.display.Video object>'}
#####
Found video at ../assets/dog_with_water.mp4
No base64-encoded video data found in HTML string.


In [5]:
%%capture_video -p "embedded.mp4"
# ✅✅✅
Video("../assets/dog_with_water.mp4",embed=True,width=300)

{'text/html': '<video controls  width="300" >\n'
              ' <source '
              'src="..." '
              'type="video/mp4">\n'
              ' Your browser does not support the video tag.\n'
              ' </video>',
 'text/plain': '<IPython.core.display.Video object>'}
#####
No video source found in HTML string.
Found base64-encoded video


In [6]:
%%capture_video -p "web.mp4"
# ❌❌❌
Video("https://github.com/Octoframes/jupyter_capture_output/blob/f8968519e9ae161c56f126f8d2cf155795fdde86/assets/dog_with_water.mp4?raw=true",width=300)


{'text/html': '<video '
              'src="https://github.com/Octoframes/jupyter_capture_output/blob/f8968519e9ae161c56f126f8d2cf155795fdde86/assets/dog_with_water.mp4?raw=true" '
              'controls  width="300" >\n'
              '      Your browser does not support the <code>video</code> '
              'element.\n'
              '    </video>',
 'text/plain': '<IPython.core.display.Video object>'}
#####
Found video at https://github.com/Octoframes/jupyter_capture_output/blob/f8968519e9ae161c56f126f8d2cf155795fdde86/assets/dog_with_water.mp4?raw=true


FileNotFoundError: [Errno 2] No such file or directory: 'https:/github.com/Octoframes/jupyter_capture_output/blob/f8968519e9ae161c56f126f8d2cf155795fdde86/assets/dog_with_water.mp4?raw=true'

In [ ]:
### maybe useful at one point in future
# from IPython.display import Video
# import requests

# url = "https://github.com/Octoframes/jupyter_capture_output/blob/f8968519e9ae161c56f126f8d2cf155795fdde86/assets/dog_with_water.mp4?raw=true"
# filename = "dog_with_water.mp4"

# response = requests.get(url)

# with open(filename, "wb") as f:
#     f.write(response.content)
# #

In [7]:
from pathlib import Path
Path("not_embedded.mp4").unlink()
Path("embedded.mp4").unlink()

try:
    Path("web.mp4").unlink()
except:
    pass